# Instructions
1. Go to https://colab.research.google.com and choose the \"Upload\" option to upload this notebook file.
1. In the Edit menu, choose \"Notebook Settings\" and then set the \"Hardware Accelerator\" dropdown to GPU.
1. Read through the code in the following sections:
  * [IMDB Dataset](#scrollTo=NPa7eLiaaof0)
  * [Define Model](#scrollTo=ihsQ5xEoaog6)
  * [Train Model](#scrollTo=OlXYR7KNaohE)
  * [Assess Model](#scrollTo=LkS3AAQraohK)
1. Complete at least one of these exercises. Remember to keep notes about what you do!
  * [Exercise Option #1 - Standard Difficulty](#scrollTo=VU4-GCUxaohS)
  * [Exercise Option #2 - Advanced Difficulty](#scrollTo=VU4-GCUxaohS)

## Documentation/Sources
* [https://radimrehurek.com/gensim/models/word2vec.html](https://radimrehurek.com/gensim/models/word2vec.html) for more information about how to use gensim word2vec in general
* _Blog post has been removed_ [https://codekansas.github.io/blog/2016/gensim.html](https://codekansas.github.io/blog/2016/gensim.html) for information about using it to create embedding layers for neural networks.
* [https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/](https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/) for information on sequence classification with keras
* [https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html) for using pre-trained embeddings with keras (though the syntax they use for the model layers is different than most other tutorials).
* [https://keras.io/](https://keras.io/) Keras API documentation

In [5]:
# upgrade tensorflow to tensorflow 2
%tensorflow_version 2.x
# display matplotlib plots
%matplotlib inline

# IMDB Dataset
The [IMDB dataset](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification) consists of movie reviews that have been marked as positive or negative. (There is also a built-in dataset of [Reuters newswires](https://keras.io/datasets/#reuters-newswire-topics-classification) that have been classified by topic.)

In [6]:
from keras.datasets import imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data()

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

It looks like our labels consist of 0 or 1, which makes sense for positive and negative.

In [7]:
print(y_train[0:9])
print(max(y_train))
print(min(y_train))

[1 0 0 1 0 0 1 0 1]
1
0


But x is a bit more trouble. The words have already been converted to numbers -- numbers that have nothing to do with the word embeddings we spent time learning!

In [8]:
x_train[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 22665,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 21631,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 19193,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 10311,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 31050,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 12118,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5

Looking at the help page for imdb, it appears there is a way to get the word back. Phew.

In [9]:
help(imdb)

Help on module keras.datasets.imdb in keras.datasets:

NAME
    keras.datasets.imdb - IMDB sentiment classification dataset.

FILE
    /usr/local/lib/python3.7/dist-packages/keras/datasets/imdb.py




In [10]:
imdb_offset = 3
imdb_map = dict((index + imdb_offset, word) for (word, index) in imdb.get_word_index().items())
imdb_map[0] = 'PADDING'
imdb_map[1] = 'START'
imdb_map[2] = 'UNKNOWN'

1646592/1641221 [==============================] - 0s 0us/step


The knowledge about the initial indices and offset came from [this stack overflow post](https://stackoverflow.com/questions/42821330/restore-original-text-from-keras-s-imdb-dataset) after I got gibberish when I tried to translate the first review, below. It looks coherent now!

In [11]:
' '.join([imdb_map[word_index] for word_index in x_train[0]])

"START this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert redford's is an amazing actor and now the same being director norman's father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the part's of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and shou

For this exercise, we're going to keep all inputs the same length (we'll see how to do variable-length later). This means we need to choose a maximum length for the review, cutting off longer ones and adding padding to shorter ones. What should we make the length? Let's understand our data.

In [12]:
lengths = [len(review) for review in x_train + x_test]
print('Longest review: {} Shortest review: {}'.format(max(lengths), min(lengths)))


Longest review: 2697 Shortest review: 70


2697 words! Wow. Well, let's see how many reviews would get cut off at a particular cutoff.

In [13]:
cutoff = 500
print('{} reviews out of {} are over {}.'.format(
    sum([1 for length in lengths if length > cutoff]), 
    len(lengths), 
    cutoff))

8485 reviews out of 25000 are over 500.


In [14]:
from keras.preprocessing import sequence
x_train_padded = sequence.pad_sequences(x_train, maxlen=cutoff)
x_test_padded = sequence.pad_sequences(x_test, maxlen=cutoff)

# Define Model

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, Dense, Flatten, Dropout
from tensorflow import test
from tensorflow import device

The embedding layer here learns the 100-dimensional vector embedding within the overall classification problem training. That is usually what we want, unless we have a bunch of un-tagged data that could be used to train word vectors but not a classification model.

In [21]:
not_pretrained_model = Sequential()
not_pretrained_model.add(Embedding(input_dim=len(imdb_map), output_dim=100, input_length=cutoff))
not_pretrained_model.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
not_pretrained_model.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
not_pretrained_model.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
not_pretrained_model.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
not_pretrained_model.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
not_pretrained_model.add(Flatten())
not_pretrained_model.add(Dense(units=128, activation='relu'))
not_pretrained_model.add(Dense(units=1, activation='sigmoid')) # because at the end, we want one yes/no answer
not_pretrained_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

# Train Model

In [22]:
# Train using GPU acceleration
# (see https://colab.research.google.com/notebooks/gpu.ipynb#scrollTo=Y04m-jvKRDsJ)
device_name = test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

with device('/device:GPU:0'):
  not_pretrained_model.fit(x_train_padded, y_train, epochs=1, batch_size=64)

391/391 [==============================] - 39s 97ms/step - loss: 0.5712 - binary_accuracy: 0.6333


# Assess Model

In [24]:
with device('/device:GPU:0'):
  not_pretrained_scores = not_pretrained_model.evaluate(x_test_padded, y_test)
print('loss: {} accuracy: {}'.format(*not_pretrained_scores))

782/782 [==============================] - 3s 4ms/step - loss: 0.3103 - binary_accuracy: 0.8700
loss: 0.31027108430862427 accuracy: 0.8700399994850159


# Exercise Option #1 - Standard Difficulty
Changing different hyperparameters of the model and observing how the performance changes.

In [3]:
# So that I do not have to repeatedly write the same code
def train_model(model):
  device_name = test.gpu_device_name()
  if device_name != '/device:GPU:0':
    print(
        '\n\nThis error most likely means that this notebook is not '
        'configured to use a GPU.  Change this in Notebook Settings via the '
        'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
    raise SystemError('GPU device not found')

  with device('/device:GPU:0'):
    model.fit(x_train_padded, y_train, epochs=2, batch_size=64) # I have increased it to 2 epochs

def evaluate_model(model):
  with device('/device:GPU:0'):
    model_scores = model.evaluate(x_test_padded, y_test)
  print('loss: {} accuracy: {}'.format(*model_scores))

In [43]:
# no changes

first_tuning = Sequential()
first_tuning.add(Embedding(input_dim=len(imdb_map), output_dim=100, input_length=cutoff))
first_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
first_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
first_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
first_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
first_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
first_tuning.add(Flatten())
first_tuning.add(Dense(units=128, activation='relu'))
first_tuning.add(Dense(units=1, activation='sigmoid')) # because at the end, we want one yes/no answer
first_tuning.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

In [44]:
train_model(first_tuning)
evaluate_model(first_tuning)

Epoch 1/2
391/391 [==============================] - 38s 94ms/step - loss: 0.6806 - binary_accuracy: 0.5127
Epoch 2/2
782/782 [==============================] - 3s 4ms/step - loss: 0.2953 - binary_accuracy: 0.8830
loss: 0.2953038811683655 accuracy: 0.8830400109291077


In [25]:
# one less conv layer

second_tuning = Sequential()
second_tuning.add(Embedding(input_dim=len(imdb_map), output_dim=100, input_length=cutoff))
second_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
second_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
second_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
second_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
#second_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu')) # got rid of a conv layer
second_tuning.add(Flatten())
second_tuning.add(Dense(units=128, activation='relu'))
second_tuning.add(Dense(units=1, activation='sigmoid')) # because at the end, we want one yes/no answer
second_tuning.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

In [26]:
train_model(second_tuning)
evaluate_model(second_tuning)

Epoch 1/2
391/391 [==============================] - 38s 96ms/step - loss: 0.6106 - binary_accuracy: 0.5931
Epoch 2/2
782/782 [==============================] - 3s 4ms/step - loss: 0.2947 - binary_accuracy: 0.8822
loss: 0.2946929931640625 accuracy: 0.8822399973869324


In [17]:
# one more conv layer

third_tuning = Sequential()
third_tuning.add(Embedding(input_dim=len(imdb_map), output_dim=100, input_length=cutoff))
third_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
third_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
third_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
third_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
third_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
third_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu')) # added additional conv layer
third_tuning.add(Flatten())
third_tuning.add(Dense(units=128, activation='relu'))
third_tuning.add(Dense(units=1, activation='sigmoid')) # because at the end, we want one yes/no answer
third_tuning.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

In [18]:
train_model(third_tuning)
evaluate_model(third_tuning)

Epoch 1/2
391/391 [==============================] - 72s 98ms/step - loss: 0.5898 - binary_accuracy: 0.6102
Epoch 2/2
782/782 [==============================] - 4s 4ms/step - loss: 0.3130 - binary_accuracy: 0.8768
loss: 0.31300050020217896 accuracy: 0.876800000667572


In [29]:
# two more conv layers 

fourth_tuning = Sequential()
fourth_tuning.add(Embedding(input_dim=len(imdb_map), output_dim=100, input_length=cutoff))
fourth_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
fourth_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
fourth_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
fourth_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
fourth_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
fourth_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu')) # added additional conv layer
fourth_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu')) # added additional conv layer
fourth_tuning.add(Flatten())
fourth_tuning.add(Dense(units=128, activation='relu'))
fourth_tuning.add(Dense(units=1, activation='sigmoid')) # because at the end, we want one yes/no answer
fourth_tuning.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

In [30]:
train_model(fourth_tuning)
evaluate_model(fourth_tuning)

Epoch 1/2
391/391 [==============================] - 39s 98ms/step - loss: 0.5613 - binary_accuracy: 0.6469
Epoch 2/2
782/782 [==============================] - 3s 4ms/step - loss: 0.3225 - binary_accuracy: 0.8723
loss: 0.32250797748565674 accuracy: 0.8723199963569641


In [39]:
# dropout layer + one more conv layer

fifth_tuning = Sequential()
fifth_tuning.add(Embedding(input_dim=len(imdb_map), output_dim=100, input_length=cutoff))
fifth_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
fifth_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
fifth_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
fifth_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
fifth_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
fifth_tuning.add(Dropout(0.15)) # added dropout layer
fifth_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu')) # added additional conv layer
fifth_tuning.add(Flatten())
fifth_tuning.add(Dense(units=128, activation='relu'))
fifth_tuning.add(Dense(units=1, activation='sigmoid')) # because at the end, we want one yes/no answer
fifth_tuning.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

In [40]:
train_model(fifth_tuning)
evaluate_model(fifth_tuning)

Epoch 1/2
391/391 [==============================] - 39s 99ms/step - loss: 0.6785 - binary_accuracy: 0.5191
Epoch 2/2
782/782 [==============================] - 3s 4ms/step - loss: 0.3103 - binary_accuracy: 0.8720
loss: 0.31033948063850403 accuracy: 0.871999979019165


In [41]:
# dropout layer + two more conv layers

sixth_tuning = Sequential()
sixth_tuning.add(Embedding(input_dim=len(imdb_map), output_dim=100, input_length=cutoff))
sixth_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
sixth_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
sixth_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
sixth_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
sixth_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
sixth_tuning.add(Dropout(0.15)) # added dropout layer
sixth_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu')) # added additional conv layer
sixth_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu')) # added additional conv layer
sixth_tuning.add(Flatten())
sixth_tuning.add(Dense(units=128, activation='relu'))
sixth_tuning.add(Dense(units=1, activation='sigmoid')) # because at the end, we want one yes/no answer
sixth_tuning.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

In [42]:
train_model(sixth_tuning)
evaluate_model(sixth_tuning)

Epoch 1/2
391/391 [==============================] - 39s 98ms/step - loss: 0.6261 - binary_accuracy: 0.5732
Epoch 2/2
782/782 [==============================] - 3s 4ms/step - loss: 0.3392 - binary_accuracy: 0.8707
loss: 0.33917686343193054 accuracy: 0.8707200288772583


In [19]:
# two dropout layers + two more conv layers

seventh_tuning = Sequential()
seventh_tuning.add(Embedding(input_dim=len(imdb_map), output_dim=100, input_length=cutoff))
seventh_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
seventh_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
seventh_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
seventh_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
seventh_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
seventh_tuning.add(Dropout(0.15)) # added dropout layer
seventh_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu')) # added additional conv layer
seventh_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu')) # added additional conv layer
seventh_tuning.add(Flatten())
seventh_tuning.add(Dense(units=128, activation='relu'))
seventh_tuning.add(Dense(units=1, activation='sigmoid')) # because at the end, we want one yes/no answer
seventh_tuning.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

In [20]:
train_model(seventh_tuning)
evaluate_model(seventh_tuning)

Epoch 1/2
391/391 [==============================] - 40s 99ms/step - loss: 0.6144 - binary_accuracy: 0.5888
Epoch 2/2
782/782 [==============================] - 3s 4ms/step - loss: 0.3072 - binary_accuracy: 0.8763
loss: 0.30723750591278076 accuracy: 0.8763200044631958


In [25]:
# two less conv layer

eigth_tuning = Sequential()
eigth_tuning.add(Embedding(input_dim=len(imdb_map), output_dim=100, input_length=cutoff))
eigth_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
eigth_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
eigth_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
#eigth_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu')) # got rid of a conv layer
#eigth_tuning.add(Conv1D(filters=32, kernel_size=5, activation='relu')) # got rid of a conv layer
eigth_tuning.add(Flatten())
eigth_tuning.add(Dense(units=128, activation='relu'))
eigth_tuning.add(Dense(units=1, activation='sigmoid')) # because at the end, we want one yes/no answer
eigth_tuning.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

In [26]:
train_model(eigth_tuning)
evaluate_model(eigth_tuning)

Epoch 1/2
391/391 [==============================] - 37s 94ms/step - loss: 0.5409 - binary_accuracy: 0.6702
Epoch 2/2
782/782 [==============================] - 3s 4ms/step - loss: 0.3747 - binary_accuracy: 0.8634
loss: 0.3746694028377533 accuracy: 0.8633999824523926
